In [7]:
################################## LIBRARIES ################################## 

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_distances 
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn_extra.cluster import KMedoids

from bertopic.dimensionality import BaseDimensionalityReduction        
from bertopic.cluster import BaseCluster
from bertopic import BERTopic

from nltk.tokenize import WhitespaceTokenizer

from ast import literal_eval
from pathlib import Path
from umap import UMAP
import pandas as pd
import numpy as np
import regex as re
import DrainMethod
import contextlib
import pickle
import sys
import os

import matplotlib.pyplot as plt
import seaborn as sns

from rouge import Rouge

############################## AUXILIARY METHODS ############################## 

# Code for reading HuggingFace token
def get_huggingface_token():
    f = open("huggingface_token.txt", "r")
    return (f.read())

# Calls conversion from data to dataframe
def load_data():
    headers, regex = generate_logformat_regex(log_format)
    return log_to_dataframe(os.path.join(indir, logName), regex, headers, log_format)

# Preprocesses dataframe with regexes, if necessary - more preprocessing to add
def preprocess_df(df_log):
    for idx, content in df_log["Content"].items():
        for currentRex in regex:
            df_log.at[idx,'Content'] = re.sub(currentRex, '<*>', content)
    return df_log

# Function to generate regular expression to split log messages
def generate_logformat_regex(log_format):
    headers = []
    splitters = re.split(r'(<[^<>]+>)', log_format)
    regex = ''
    for k in range(len(splitters)):
        if k % 2 == 0:
            splitter = re.sub(' +', '\\\s+', splitters[k])
            regex += splitter
        else:
            header = splitters[k].strip('<').strip('>')
            regex += f'(?P<{header}>.*?)'
            headers.append(header)
    regex = re.compile('^' + regex + '$')
    return headers, regex

# Function to transform log file to dataframe 
def log_to_dataframe(log_file, regex, headers, logformat):
    log_messages = []
    linecount = 0
    with open(log_file, 'r') as fin:
        for line in fin.readlines():
            with contextlib.suppress(Exception):
                match = regex.search(line.strip())
                message = [match.group(header) for header in headers]
                log_messages.append(message)
                linecount += 1
    logdf = pd.DataFrame(log_messages, columns=headers)
    logdf.insert(0, 'LineId', None)
    logdf['LineId'] = [i + 1 for i in range(linecount)]
    return logdf

# Transforms the dataset, creating raw vector file
def transform_dataset(raw_content):
    
    path_to_file = os.path.join(vector_dir, logName + '_vectors_TFIDF.vec')
    path = Path(path_to_file)
    vectors_tfidf = []

    if (path.is_file()):
        vectors_tfidf = pickle.load(open(path_to_file, 'rb'))
    else:
        #Using TFIDF Vectorizer 
        #print("Starting encode")
        tr_idf_model  = TfidfVectorizer()
        vectors_tfidf = tr_idf_model.fit_transform(raw_content)
        pickle.dump(vectors_tfidf, open(path_to_file, 'wb'))
    
    # # Using TFIDF Vectorizer 
    # tr_idf_model  = TfidfVectorizer()
    # vectors_tfidf = tr_idf_model.fit_transform(raw_content)
    # pickle.dump(vectors_tfidf, open(path_to_file, 'wb'))

    return vectors_tfidf

def creates_lists(clusterer):

    ## General Parameters
    cluster_idxs = []
    cluster_lines = []
    output_dir = os.path.join(os.getcwd(), "results")  # The output directory of parsing results
    output_csv = os.path.join(output_dir, log_file + '_structured.csv') 

    ## Code

    # Reads parameters list
    full_df = pd.read_csv(output_csv)
    elem_df = full_df["EventTemplate"]

    # Creates blank lists
    for elem in range (clusterer.labels_.max()+1):
        cluster_idxs.append([])
        cluster_lines.append([])

    # Populate the lists with cluster elements
    for idx, elem in np.ndenumerate(clusterer.labels_):
        if elem != -1:
            cluster_idxs[elem].append(idx[0])
            cluster_lines[elem].append(elem_df[idx[0]])
        
    return (cluster_idxs, cluster_lines)

## Gets number of different templates using Drain
def get_template_number():

    target_file = "results/" + dataset + "_lines.txt_templates.csv"
    csv = pd.read_csv(target_file)
    content = csv["EventTemplate"]
    return (len(content))


################################# MAIN METHODS ################################ 

# Code for reading HuggingFace token
def get_huggingface_token():
    f = open("huggingface_token.txt", "r")
    return (f.read())

# Parse logs using Drain
def parse_logs(log_file, st=0.5, depth=5):
    st = st # Drain similarity threshold
    depth = depth # Max depth of the parsing tree

    ## Code
    parser = DrainMethod.LogParser(log_format=log_format, indir=indir, outdir=output_dir, rex=regex, depth=depth, st=st)
    parser.parse(log_file)

    parsedresult=os.path.join(output_dir, log_file + '_structured.csv')   

# Creates embeddings for log file
def transform(logName):
    log_df = load_data()
    log_df = preprocess_df(log_df)
    return transform_dataset(log_df["Content"])

# Creates distance matrix, using Euclidean distance
def create_distance_matrix(vector_df):
    # Using Euclidean Distance between the rows of the TFIDF Matrix
    tfidf_distance = pairwise_distances(vector_df, metric="euclidean", n_jobs=-1)
    #Normalizes Distance Matrix with Min-Max
    min_val = np.min(tfidf_distance)
    max_val = np.max(tfidf_distance)
    tfidf_distance = (tfidf_distance - min_val) / (max_val - min_val)
    return (tfidf_distance)

# Creates variable matrix, using Jaccard distance
def create_variable_matrix():
    ## General Parameters
    output_dir = os.path.join(os.getcwd(), "results")  # The output directory of parsing results
    output_csv = os.path.join(output_dir, log_file + '_structured.csv') 

    ## Code
    # Reads parameters list
    full_df = pd.read_csv(output_csv)
    var_df = full_df["ParameterList"]

    # Breaks the string into lists
    for i, line in var_df.items():
        var_df.at[i] = literal_eval(var_df.at[i])

    # Transforms variable list to variable sparse matrix
    mlb = MultiLabelBinarizer(sparse_output=True)
    var_df = mlb.fit_transform(var_df)
    var_distance = pairwise_distances(np.asarray(var_df.todense()), metric="jaccard", n_jobs=-1)
    return (var_distance)

def creates_closeness_matrix(tfidf_distance):
    # Creates Count Matrix using line numbers from log lines as the counter
    count_list = []
    n = len(tfidf_distance)
    count_distance = np.zeros(shape=(n, n), dtype=int)
    for i in range(n):
            count_list.append(i)

    # Using a Subtraction Distance using the line numbers as a Count Matrix
    count_array = np.array(count_list)
    for x in count_array:
        for y in count_array:
            count_distance[x,y] = abs(x-y)
    # Normalizes Distance Matrix with Min-Max
    min_val = np.min(count_distance)
    max_val = np.max(count_distance)
    count_distance = (count_distance - min_val) / (max_val - min_val)
    return (count_distance)

def saves_matrices(distance_mat, variable_mat, closeness_mat):
    np.save("tfidf_distance_" + logName + ".csv", distance_mat)
    np.save("var_distance_" + logName + ".csv", variable_mat)
    np.save("count_distance_" + logName + ".csv", closeness_mat)

def loads_matrices():
    tfidf_distance = np.load("tfidf_distance_" + logName + ".csv")
    count_distance = np.load("count_distance_" + logName + ".csv")
    var_distance = np.load("var_distance_" + logName + ".csv") 
    return (tfidf_distance, count_distance, var_distance)

def joins_matrices(tfidf_distance, var_distance, count_distance, alpha, beta, gamma):

    if alpha+beta+gamma > 1:
        raise Exception("Values have to sum 1!")

    # New matrices, corrected by the weights
    tfidf_distance_wtd = np.dot(alpha,tfidf_distance)
    var_distance_wtd = np.dot(beta, var_distance)
    count_distance_wtd = np.dot(gamma, count_distance)

    # Sums remaining matrices
    unified_matrix = np.asarray(tfidf_distance_wtd + var_distance_wtd + count_distance_wtd)
    return (unified_matrix)

def cluster_kmedoids(unified_matrix, cluster_num):
    ## Clusters with cluster_kmedoids

    clusterer = KMedoids(n_clusters=cluster_num, method='pam', init='k-medoids++')
    clusterer.fit(unified_matrix)

    ## Checks number of outliers
    cont = np.count_nonzero(clusterer.labels_ == -1)
    return (clusterer)

def find_topics_bertopic(cluster_list, cluster_number, num_topics):
        
        empty_reduction_model = BaseDimensionalityReduction()
        empty_cluster_model = KMedoids(n_clusters = 1)
        
        topic_model = BERTopic(hdbscan_model=empty_cluster_model, umap_model=empty_reduction_model, top_n_words=10)

        #Applies BertTopic
        topics, probs = topic_model.fit_transform(cluster_list[cluster_number])

        #Gets summary of topics
        topic_model.get_topic(0)
        top_topic = topic_model.get_topic(0)
        words = [i[0] for i in top_topic]
        summary = ' '.join(words)

        return (summary)

def bertopic_previous_clustering(clusterer):
    cluster_idxs, cluster_lines = creates_lists(clusterer)
    cluster_topic = []
    topic_summaries = []

    ## Creates list of boolean values, representing summarized topics
    for idx in range(clusterer.labels_.max()):
        cluster_topic.append(None)

    for i, elem in enumerate(clusterer.labels_):

        ## For each cluster, maps topics, and defines them as the summary
        if (cluster_topic[elem-1] == None):
            summary = find_topics_bertopic(cluster_lines, elem-1, 1)
            cluster_topic[elem-1] = summary
        
        if elem == -1:
            topic_summaries.append("")
        else:
            topic_summaries.append(cluster_topic[elem-1])
        
        target_file = "ground_truths/" + dataset + "_bert_topics_tests.txt"
        with open (target_file, "w") as f:
            for line in topic_summaries:
                f.write(f"{line}\n")

    return topic_summaries

def consider_previous_clustering():
    ## Tests with BerTopic


    target_file = "ground_truths/" + dataset + "_lines.txt_structured.csv"
    csv = pd.read_csv(target_file)
    content = csv["EventTemplate"]
    num_topics = 10
    line_file = []
    line_set = []



    cluster_model = KMedoids(n_clusters=1)
    empty_reduction_model = BaseDimensionalityReduction()
    topic_model = BERTopic(hdbscan_model=cluster_model, umap_model=empty_reduction_model)

    for idx, line in enumerate(content):

        line_set.append(line + '\n')

        if (idx % 20 == 19):
    
            # print("Chegamos ao idx {}".format(idx))
            # print(line_set)

            #Applies BertTopic
            topics, probs = topic_model.fit_transform(line_set)

            #Gets summary of topics
            topic_model.get_topic(0)
            top_topic = topic_model.get_topic(0)
            words = [i[0] for i in top_topic]
            summary = ' '.join(words)

            #Finds most representative line inside the cluster
            best_line = find_best_line(line_set, summary)

            for num in range(20):
                line_file.append(summary)

            line_set = []

    ## Writes external file with created topics
    with open ("ground_truths/" + dataset + "_bert_topics.txt", "w") as f:
        for line in line_file:
            f.write(f"{line}\n")
    
    return line_file

def create_new_bertopic_model(cluster_num=8):
    
    lines = []

    #with open('ground_truths/' + dataset + '_split.txt', 'r') as line_file:
    with open('ground_truths/' + dataset + '_lines.txt', 'r') as line_file:
        

        for line in line_file:
            lines.append(line)

    umap_model = UMAP(init='random')
    hdbscan_model = KMedoids(n_clusters=cluster_num, method='pam', init='random')
    vectorizer_model = CountVectorizer(stop_words="english")

    topic_model = BERTopic(umap_model=umap_model, hdbscan_model=hdbscan_model, vectorizer_model=vectorizer_model, top_n_words=10)
    
    topics, probs = topic_model.fit_transform(lines)
    return (topic_model)

def bertopic_new_clustering(cluster_num = 8):

    topic_model = create_new_bertopic_model(cluster_num = cluster_num)
    cluster_topic = []
    topic_summaries = []

    for elem in topic_model.topics_:
        
        line_topic = topic_model.get_topic(elem)
        words = [i[0] for i in line_topic]
        summary = ' '.join(words)
        topic_summaries.append(summary)


    target_file = "ground_truths/" + dataset + "_bert_topics_tests.txt"

    ## Writes external file with created topics
    with open (target_file, "w") as f:
        for line in topic_summaries:
            f.write(f"{line}\n")

    return topic_summaries

## Method to find the most representative line inside the cluster
## raw_lines = list of lines inside LogSummary's cluster
## word_list = list of tokens composed by the LDA/BertTopic
def find_best_line(raw_lines, word_list):

    tk = WhitespaceTokenizer()

    closest_line = 0
    similar_tokens = 0
    max_similarity = 0
    for idx, line in enumerate(raw_lines):
        tokenized_line = tk.tokenize(line.lower())
        for token in tokenized_line:
            if token in word_list:
                similar_tokens += 1
        #print ("Line {} has {} identical tokens".format(idx, similar_tokens))
        if similar_tokens > max_similarity:
           max_similarity = similar_tokens
           closest_line = idx
        similar_tokens = 0
    return (raw_lines[closest_line])        

def calculates_metrics():
    
    rouge = Rouge()

    count_precision = 0
    count_recall = 0
    count_f1 = 0
    total_lines = 2000 # Number of lines

    target_file = "_bert_topics_tests.txt"

    # Opens external files with ground truth summaries and created topics
    #with open('ground_truths/' + dataset + '_split_summaries.txt', 'r') as summaries, \
    with open('ground_truths/' + dataset + '_summaries.txt', 'r') as summaries, \
        open('ground_truths/' + dataset + target_file, 'r') as topics:
        for line_summary, line_topic in zip(summaries, topics):
            line_summary = line_summary[:-2]
            line_summaries = line_summary.split(";")

            for summary in line_summaries:
                current_precision = 0
                current_recall = 0
                current_f1 = 0
                metrics = rouge.get_scores(line_topic, summary)[0]['rouge-1']  

                ## If the summary improves the f1 score, saves its metrics
                if (current_f1 < metrics['f']):
                    current_precision = metrics['p']
                    current_recall = metrics['r']
                    current_f1 = metrics['f']
            
            count_precision += current_precision
            count_recall += current_recall        
            count_f1 += current_f1

    final_precision = count_precision/total_lines
    final_recall = count_recall/total_lines
    final_f1 = count_f1/total_lines

    final = "The precision is {}, the recall is {}, the f1 score is {}".format(final_precision, final_recall, final_f1)
    print (final)
    return (final_f1)

################################# TEST SCENARIOS ################################ 

# # ## Cenário A
# # ## Testa usando clusters pré-definidos, e usando BerTopic sem clusterizar
# def tests_scenario_A(drain_st, drain_depth):

#     parameters = ("Testing scenario A using raw data matrix and predefined clustering, with drain st {}, drain depth {}".
#           format(drain_st, drain_depth))
#     print(parameters)
    
#     #parse_logs(drain_st, drain_depth)

#     consider_previous_clustering()

#     final = calculates_metrics()
#     print("F1 score: {}".format(final))

#     return (final)

## Cenário B
## Testando usando BerTopic para clusterizar, sem considerar matriz unificada, transformando os dados brutos
def tests_scenario_B(drain_st, drain_depth, n_clusters):
    
    #n_clusters = get_template_number()
    
    # parameters = ("Testing scenario B using raw data matrix and BerTopic K-Medoids clustering, drain st {}, drain depth {}, cluster number {}".
    #       format(drain_st, drain_depth, n_clusters))
    # print(parameters)

    # Runs BerTopic
    #parse_logs(drain_st, drain_depth)
    topic_summaries = bertopic_new_clustering(n_clusters)

    final = calculates_metrics()
    #print("F1 score: {}".format(final))

    return (final)

## Cenário C
## Testando usando transformação via matriz unificada, depois BerTopic para seleção de tópicos
def tests_scenario_C(drain_st, drain_depth, alpha, beta, gamma, n_clusters=8):

    #parse_logs(drain_st, drain_depth)
    #n_clusters = get_template_number()

    parameters = ("Testing scenario C using joint matrix and BerTopic topic modeling, with drain st {}, drain depth {}, alpha {}, beta {}, gamma {}, cluster number {}".
          format(drain_st, drain_depth, alpha, beta, gamma, n_clusters))
    print(parameters)

    # Criação matriz unificada
    vector_df = transform(os.path.basename(logName))

    distance_matrix = create_distance_matrix(vector_df)
    variable_matrix = create_variable_matrix()
    closeness_matrix = creates_closeness_matrix(distance_matrix)

    joint_matrix = joins_matrices(distance_matrix, variable_matrix, closeness_matrix, 
                                alpha, beta, gamma)
      
    clusterer = cluster_kmedoids (joint_matrix, n_clusters)

    topic_summaries = bertopic_previous_clustering(clusterer)
    
    final = calculates_metrics()
    #print("F1 score: {}".format(final))

    return (final)

def run_tests(results, dataset, drain_st, drain_depth, num_executions, n_clusters):

    #Variable Matrix Parameters
    alpha = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
    beta = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
    gamma = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
    # alpha = [0.3]
    # beta = [0.4, 0.5]
    # gamma = [0.2, 0.3]

    # # # Running Single Test for Scenario A
    # value = tests_scenario_A(drain_st, drain_depth)
    # new_row = [dataset, 'A', drain_st, drain_depth, 0, 0, 0, value]
    # results.loc[len(results)] = new_row

    # # Running num_executions for Scenario B
    # for i in range(num_executions):
    #     value = tests_scenario_B(drain_st, drain_depth, n_clusters)
    #     new_row = [dataset, 'B', drain_st, drain_depth, 0, 0, 0, value]
    #     results.loc[len(results)] = new_row

    # # Testing Different Hyperparameters for Scenario C
    best_f1 = 0
    best_alpha = 0
    best_beta = 0
    best_gamma = 0
    for a in alpha:
        for b in beta:
            for g in gamma:
                if (a+b+g != 1):
                    pass
                else:
                    for i in range(num_executions):
                        try:
                            value = tests_scenario_C(drain_st, drain_depth, a, b, g, n_clusters)
                            #print(value)
                            new_row = [dataset, 'C', drain_st, drain_depth, a, b, g, value]
                            results.loc[len(results)] = new_row
                        except Exception as error:
                            value = 0
                            print(error)
                        if (value > best_f1):
                            best_f1 = value
                            best_alpha = a
                            best_beta = b
                            best_gamma = g
    print("A melhor combinação foi alpha {}, beta {}, gamma {}".format(best_alpha, best_beta, best_gamma))

In [5]:
# bgl
## General parameters 
input_dir = os.path.join(os.getcwd(), "ground_truths") # The input directory of raw logs
output_dir = os.path.join(os.getcwd(), "results")  # The output directory of parsing results
vector_dir = os.path.join(os.getcwd(), "vectors")  # The vector directory of converted logs
dataset = "bgl"
drain_st = 0.3
drain_depth = 3
logName = dataset + '_lines.txt' # Name of file to be parsed
log_format = '<Content>' # Format of the file, if there are different fields
regex = [] # Regex strings for Drain execution
indir = os.path.join(input_dir, os.path.dirname(logName))
log_file = os.path.basename(logName)

parse_logs(log_file, drain_st, drain_depth)
tests_scenario_C(drain_st, drain_depth, 0.1, 0.6, 0.3)
tests_scenario_C(drain_st, drain_depth, 0.7, 0.2, 0.1)

Testing scenario C using joint matrix and BerTopic topic modeling, with drain st 0.3, drain depth 3, alpha 0.1, beta 0.6, gamma 0.3, cluster number 53
The precision is 0.030028571428571616, the recall is 0.05955000000000028, the f1 score is 0.03885476109236957
Testing scenario C using joint matrix and BerTopic topic modeling, with drain st 0.3, drain depth 3, alpha 0.7, beta 0.2, gamma 0.1, cluster number 53
The precision is 0.018377777777777788, the recall is 0.03283333333333337, the f1 score is 0.021796702887161745


0.021796702887161745

In [8]:
tests_scenario_C(drain_st, drain_depth, 0.1, 0.6, 0.3, 8)
tests_scenario_C(drain_st, drain_depth, 0.7, 0.2, 0.1, 8)

Testing scenario C using joint matrix and BerTopic topic modeling, with drain st 0.3, drain depth 3, alpha 0.1, beta 0.6, gamma 0.3, cluster number 8
The precision is 0.019500000000000108, the recall is 0.0528333333333336, the f1 score is 0.02752380877535422
Testing scenario C using joint matrix and BerTopic topic modeling, with drain st 0.3, drain depth 3, alpha 0.7, beta 0.2, gamma 0.1, cluster number 8
The precision is 0.054944444444444705, the recall is 0.12549999999999936, the f1 score is 0.07425124758818019


0.07425124758818019

In [3]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', token=get_huggingface_token())
embeddings = model.encode(sentences)
print(embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[ 6.76569417e-02  6.34959564e-02  4.87131029e-02  7.93048963e-02
   3.74480709e-02  2.65277619e-03  3.93749177e-02 -7.09846616e-03
   5.93614057e-02  3.15370224e-02  6.00980818e-02 -5.29052354e-02
   4.06067409e-02 -2.59308442e-02  2.98428535e-02  1.12691487e-03
   7.35148937e-02 -5.03818579e-02 -1.22386597e-01  2.37028375e-02
   2.97265276e-02  4.24768776e-02  2.56337691e-02  1.99514348e-03
  -5.69190495e-02 -2.71597989e-02 -3.29035297e-02  6.60248995e-02
   1.19007193e-01 -4.58791330e-02 -7.26214200e-02 -3.25840116e-02
   5.23414016e-02  4.50553410e-02  8.25305004e-03  3.67024243e-02
  -1.39415674e-02  6.53918684e-02 -2.64272075e-02  2.06393175e-04
  -1.36643527e-02 -3.62811014e-02 -1.95043907e-02 -2.89738420e-02
   3.94270532e-02 -8.84090662e-02  2.62423488e-03  1.36713171e-02
   4.83062454e-02 -3.11566181e-02 -1.17329188e-01 -5.11690676e-02
  -8.85287672e-02 -2.18963325e-02  1.42986281e-02  4.44167778e-02
  -1.34816021e-02  7.43392631e-02  2.66382936e-02 -1.98762678e-02
   1.79191

In [ ]:
################################## RUNNING TESTS ################################## 

## General parameters 
input_dir = os.path.join(os.getcwd(), "ground_truths") # The input directory of raw logs
output_dir = os.path.join(os.getcwd(), "results")  # The output directory of parsing results
vector_dir = os.path.join(os.getcwd(), "vectors")  # The vector directory of converted logs
results = pd.DataFrame(columns=['Dataset','Scenario', 'Drain St', 'Drain Depth', 'Alpha', 'Beta', 'Gamma', 'F1'])
num_executions = 10

# Testing BGL
dataset = "bgl" # The name of the dataset being tested
drain_st = 0.3
drain_depth = 3
n_clusters = 49
#logName = dataset + '_lines.txt' # Name of file to be parsed
logName = dataset + "_split.txt" # Name of the sliced file for training
log_format = '<Content>' # Format of the file, if there are different fields
regex = [] # Regex strings for Drain execution
indir = os.path.join(input_dir, os.path.dirname(logName))
log_file = os.path.basename(logName)
#parse_logs(log_file, drain_st, drain_depth)
run_tests(results, dataset, drain_st, drain_depth, num_executions, n_clusters)

# Saves temporary CSV
results.to_csv('testing_results.csv', index=False)  

# Testing HDFS
dataset = "hdfs" # The name of the dataset being tested
drain_st = 0.3
drain_depth = 5
n_clusters = 12
#logName = dataset + '_lines.txt' # Name of file to be parsed
logName = dataset + "_split.txt" # Name of the sliced file for training
log_format = '<Content>' # Format of the file, if there are different fields
regex = [] # Regex strings for Drain execution
indir = os.path.join(input_dir, os.path.dirname(logName))
log_file = os.path.basename(logName)
#parse_logs(log_file, drain_st, drain_depth)
run_tests(results, dataset, drain_st, drain_depth, num_executions, n_clusters)

# Saves temporary CSV
results.to_csv('testing_results.csv', index=False)  

# Testing HPC
dataset = "hpc" # The name of the dataset being tested
drain_st = 0.4
drain_depth = 4
n_clusters = 34
#logName = dataset + '_lines.txt' # Name of file to be parsed
logName = dataset + "_split.txt" # Name of the sliced file for training
log_format = '<Content>' # Format of the file, if there are different fields
regex = [] # Regex strings for Drain execution
indir = os.path.join(input_dir, os.path.dirname(logName))
log_file = os.path.basename(logName)
#parse_logs(log_file, drain_st, drain_depth)
run_tests(results, dataset, drain_st, drain_depth, num_executions, n_clusters)

# Saves temporary CSV
results.to_csv('testing_results.csv', index=False)  

# Testing Proxifier
dataset = "Proxifier" # The name of the dataset being tested
drain_st = 0.3
drain_depth = 4
n_clusters = 33
#logName = dataset + '_lines.txt' # Name of file to be parsed
logName = dataset + "_split.txt" # Name of the sliced file for training
log_format = '<Content>' # Format of the file, if there are different fields
regex = [] # Regex strings for Drain execution
indir = os.path.join(input_dir, os.path.dirname(logName))
log_file = os.path.basename(logName)
#parse_logs(log_file, drain_st, drain_depth)
run_tests(results, dataset, drain_st, drain_depth, num_executions, n_clusters)

# Saves temporary CSV
results.to_csv('testing_results.csv', index=False)  

# Testing Spark
dataset = "spark" # The name of the dataset being tested
drain_st = 0.3
drain_depth = 3
n_clusters = 32
#logName = dataset + '_lines.txt' # Name of file to be parsed
logName = dataset + "_split.txt" # Name of the sliced file for training
log_format = '<Content>' # Format of the file, if there are different fields
regex = [] # Regex strings for Drain execution
indir = os.path.join(input_dir, os.path.dirname(logName))
log_file = os.path.basename(logName)
#parse_logs(log_file, drain_st, drain_depth)
run_tests(results, dataset, drain_st, drain_depth, num_executions, n_clusters)

# Saves temporary CSV
results.to_csv('testing_results.csv', index=False)  

# Testing Zookeeper
dataset = "Zookeeper" # The name of the dataset being tested
drain_st = 0.3
drain_depth = 3
n_clusters = 35
#logName = dataset + '_lines.txt' # Name of file to be parsed
logName = dataset + "_split.txt" # Name of the sliced file for training
log_format = '<Content>' # Format of the file, if there are different fields
regex = [] # Regex strings for Drain execution
indir = os.path.join(input_dir, os.path.dirname(logName))
log_file = os.path.basename(logName)
#parse_logs(log_file, drain_st, drain_depth)
run_tests(results, dataset, drain_st, drain_depth, num_executions, n_clusters)

# Saves final CSV
results.to_csv('testing_results.csv', index=False)  

A melhor combinação foi alpha 0.1, beta 0.6, gamma 0.3
A melhor combinação foi alpha 0, beta 1, gamma 0
